In [1]:
import geopandas, folium, sqlite3, shapely.wkt, os, json
from shapely.geometry import Polygon, MultiPolygon
from folium import plugins
import shapely.ops as so
import pandas as pd
import numpy as np

os.chdir('..')
db = '____TIF_Calc_DB____.sqlite'

active_bras =  KEY FILE HERE

plans = pd.read_csv(active_bras)
plans.head()

,Plan,Slot,FileName
0,10108 W SEVEN MILE,BRA1,TY2018 - 10108 W SEVEN MILE
1,1475 E JEFFERSON-17,BRA1,TY2018 - 1475 E JEFFERSON-17
2,3100 WOODWARD - 05,BRA1,TY2018 - 3100 WOODWARD - 05
3,3408 WOODWARD-05,BRA1,TY2018 - 3408 WOODWARD-05
4,3800 WOODWARD -10,BRA1,TY2018 - 3800 WOODWARD -10


In [2]:
# Method 3 - Scorched earth attempt to join without geo-coding

Heading = ['parcelnumber','propstreetcombined','taxable','name']

results1 = []
for plan in plans[plans.Slot =='BRA1'].Plan:
    conn = sqlite3.connect(db)
    c = conn.cursor()
    c.execute('''SELECT 
        parcelnumber,
        propstreetcombined,
        current_taxable,
        dda_tifa_code_0
    FROM ParcelDetail
    WHERE TaxYear is 2018
        AND dda_tifa_code_0 is '%s';'''%(plan))
    df = c.fetchall()
    conn.close()
    results1.extend(df)

BRA1 = pd.DataFrame(results1, columns = Heading)

results2 = []
for plan in plans[plans.Slot =='BRA3'].Plan:
    conn = sqlite3.connect(db)
    c = conn.cursor()
    c.execute('''SELECT 
        parcelnumber,
        propstreetcombined,
        current_taxable,
        dda_tifa_code_2
    FROM ParcelDetail
    WHERE TaxYear is 2018
        AND dda_tifa_code_2 is '%s';'''%(plan))
    df = c.fetchall()
    conn.close()
    results2.extend(df)

BRA3 = pd.DataFrame(results2, columns = Heading)

In [3]:
BRA1.head()

,parcelnumber,propstreetcombined,taxable,name
0,16008746-52,10130 W SEVEN MILE,301345,10108 W SEVEN MILE
1,16008753-6,10108 W SEVEN MILE,18130,10108 W SEVEN MILE
2,16991508.01,10130 W SEVEN MILE,39200,10108 W SEVEN MILE
3,16991508.00,10130 W SEVEN MILE,0,10108 W SEVEN MILE
4,05000083-99,1375 E JEFFERSON,24504,1475 E JEFFERSON-17


In [4]:
BRA3.head()

,parcelnumber,propstreetcombined,taxable,name
0,02001887-91,1075 WOODWARD AVENUE,4173001,1001 WOODWARD-04
1,02001892.,1001 WOODWARD AVENUE,1965629,1001 WOODWARD-04
2,24000207.,1001 WOODWARD AVENUE,7932,1001 WOODWARD-04
3,02991830.04,"1001 WOODWARD AVENUE 300,120",6100,1001 WOODWARD-04
4,02991830.07,1001 WOODWARD AVENUE,600,1001 WOODWARD-04


In [5]:
# Here we're really jsut trying to grab best existing polygon. Order of the OR statements is the priority we have to match.

polygons = []
for row in BRA1.itertuples():
    conn = sqlite3.connect(db)
    c = conn.cursor()
    c.execute('''Select
                    the_geom
                FROM GEOM_ASG
                WHERE 
                    PARCELNUM is '%s' OR
                    ADDRESS is '%s' OR
                    NEZ is '%s' OR
                    RELATED is '%s';'''%(row.parcelnumber, row.propstreetcombined, row.parcelnumber,row.parcelnumber))
    df = c.fetchall()
    conn.close()
    polygons.append(df)

len(polygons)==len(BRA1) ############### ABSOLUTELY MUST BE TRUE ############### !

True

In [6]:
# Add geometries to original DF, put in np.nan for null responses
BRA1['geom'] = [x[0] if len(x) > 0 else np.nan for x in polygons]
BRA1.head()

,parcelnumber,propstreetcombined,taxable,name,geom
0,16008746-52,10130 W SEVEN MILE,301345,10108 W SEVEN MILE,(MULTIPOLYGON (((-83.16303338774212 42.4317220...
1,16008753-6,10108 W SEVEN MILE,18130,10108 W SEVEN MILE,(MULTIPOLYGON (((-83.16248868169244 42.4317308...
2,16991508.01,10130 W SEVEN MILE,39200,10108 W SEVEN MILE,(MULTIPOLYGON (((-83.16303338774212 42.4317220...
3,16991508.00,10130 W SEVEN MILE,0,10108 W SEVEN MILE,(MULTIPOLYGON (((-83.16303338774212 42.4317220...
4,05000083-99,1375 E JEFFERSON,24504,1475 E JEFFERSON-17,(MULTIPOLYGON (((-83.03348444189284 42.3343938...


In [7]:
polygons = []
for row in BRA3.itertuples():
    conn = sqlite3.connect(db)
    c = conn.cursor()
    c.execute('''Select
                    the_geom
                FROM GEOM_ASG
                WHERE 
                    PARCELNUM is '%s' OR
                    ADDRESS is '%s' OR
                    NEZ is '%s' OR
                    RELATED is '%s';'''%(row.parcelnumber, row.propstreetcombined, row.parcelnumber,row.parcelnumber))
    df = c.fetchall()
    conn.close()
    polygons.append(df)

len(polygons)==len(BRA3) ############### ABSOLUTELY MUST BE TRUE ############### !

True

In [8]:
# Add geometries to original DF, put in np.nan for null responses
BRA3['geom'] = [x[0] if len(x) >0 else np.nan for x in polygons]
BRA3.head()

,parcelnumber,propstreetcombined,taxable,name,geom
0,02001887-91,1075 WOODWARD AVENUE,4173001,1001 WOODWARD-04,(MULTIPOLYGON (((-83.04825928524568 42.3326770...
1,02001892.,1001 WOODWARD AVENUE,1965629,1001 WOODWARD-04,(MULTIPOLYGON (((-83.0480418530117 42.33210535...
2,24000207.,1001 WOODWARD AVENUE,7932,1001 WOODWARD-04,(MULTIPOLYGON (((-83.0480418530117 42.33210535...
3,02991830.04,"1001 WOODWARD AVENUE 300,120",6100,1001 WOODWARD-04,NaN
4,02991830.07,1001 WOODWARD AVENUE,600,1001 WOODWARD-04,(MULTIPOLYGON (((-83.0480418530117 42.33210535...


In [9]:
#Aggregating the data
# One plan, One value, many geoms - will need to merge int multipoly
names = list(set(BRA1.name))
taxable = []
geoms = []
for item in names:
    
    # create aggregated list of tv by plan
    taxable.append(sum([int(x.taxable) for x in BRA1.itertuples() if x.name == item]))
    
    # create aggregated geometries by plan
    subList = []    
    for x in BRA1.geom[BRA1.name == item]:
        if type(x) == tuple:
            subList.append(shapely.wkt.loads(x[0]))
        
    geoms.append(so.cascaded_union(subList))

# Create list of tuples to be converted to DF
data_tuples = list(zip(names,taxable,geoms))
summarized_BRA1 = pd.DataFrame(data_tuples, columns = ['names','taxable', 'geoms'])

In [10]:
len(taxable) == len(names)

True

In [11]:
len(taxable) == len(geoms)

True

In [12]:
summarized_BRA1.head()

,names,taxable,geoms
0,EASTERN MARKET - 18,209938,(POLYGON ((-83.03968929370093 42.3418223619753...
1,HERMAN KIEFER - 17,35896,(POLYGON ((-83.09378805848102 42.3700693074953...
2,NDC PROJECT 1-02,2285356,(POLYGON ((-82.97495280788259 42.3647373978431...
3,BRUSH PARK A-B - 16,1004785,(POLYGON ((-83.05307025305186 42.3433733172446...
4,E JEFFERSON NBHD-10,789560,"POLYGON ((-83.0165443230302 42.33843522296422,..."


In [13]:
#Aggregating the data
# One plan, One value, many geoms - will need to merge int multipoly
names = list(set(BRA3.name))
taxable = []
geoms = []
for item in names:
    
    # create aggregated list of tv by plan
    taxable.append(sum([int(x.taxable) for x in BRA3.itertuples() if x.name == item]))
    
    # create aggregated geometries by plan
    subList = []    
    for x in BRA3.geom[BRA3.name == item]:
        if type(x) == tuple:
            subList.append(shapely.wkt.loads(x[0]))
        
    geoms.append(so.cascaded_union(subList))

# Create list of tuples to be converted to DF
data_tuples = list(zip(names,taxable,geoms))
summarized_BRA3 = pd.DataFrame(data_tuples, columns = ['names','taxable', 'geoms'])

In [14]:
len(taxable) == len(names)

True

In [15]:
len(taxable) == len(geoms)

True

In [16]:
summarized_BRA3.head()

,names,taxable,geoms
0,607 SHELBY - 15,1724920,POLYGON ((-83.04859254281959 42.32930068546698...
1,4830 CASS-08,3536397,POLYGON ((-83.06564909321546 42.35577037100929...
2,ASHLEY-14,2185410,"POLYGON ((-83.0477183411242 42.33643847950929,..."
3,751 GRISWOLD - 17,1051115,POLYGON ((-83.04845430229679 42.33129187047483...
4,FREE PRESS BLDG - 10,852637,"POLYGON ((-83.05121975360211 42.3301094313625,..."


In [17]:
summarized_BRA1 = summarized_BRA1.sort_values(by=['names'])
summarized_BRA3 = summarized_BRA3.sort_values(by=['names'])

In [18]:
DBRA = summarized_BRA3.append(summarized_BRA1)
DBRA.head()

,names,taxable,geoms
12,1001 WOODWARD-04,6869662,"POLYGON ((-83.0480418530117 42.33210535616158,..."
6,220 W CONGRESS - 17,291334,POLYGON ((-83.04889811838223 42.32939864377501...
1,4830 CASS-08,3536397,POLYGON ((-83.06564909321546 42.35577037100929...
0,607 SHELBY - 15,1724920,POLYGON ((-83.04859254281959 42.32930068546698...
7,640 TEMPLE - 17,4329500,POLYGON ((-83.06266781281798 42.34121935147953...


In [56]:
# Get rid of any old verion. GeoJsons do not like being overwritten
os.remove('SearchBraTest.geojson')

# Before getting into it, we'll need a highlight function
def highlight_function(feature):
    return {
        'fillColor': '#ffaf00',
        'color': 'green',
        'weight': 1.1,
    }

#Map with Folium
CENTER = [42.3318175,-83.0465565]
m = folium.Map(CENTER, zoom_start=12, tiles='cartodbpositron')

DBRA = summarized_BRA3.append(summarized_BRA1)
#__________ ALL BRA

# convert the pandas df with wkt polygons to geopandas df
gdf = geopandas.GeoDataFrame(DBRA, geometry='geoms')
gdf.crs = {'init' :'epsg:4326'}

# Write to file as geojson
gdf.to_file("SearchBraTest.geojson", driver='GeoJSON')
with open("SearchBraTest.geojson") as f:
    data = json.load(f)

# Read geojson
DBRA = folium.GeoJson(data,
        name = 'Slot 3 Brownfields',
        style_function=lambda data: {
            'fillColor': 'blue',
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.2,},
        highlight_function=highlight_function,
        tooltip=folium.GeoJsonTooltip(
        fields=['names','taxable'],
        localize=True)).add_to(m)

# Fullscreen option
plugins.Fullscreen(
    position='topleft',
    title='Expand me',
    title_cancel='Exit me',
    force_separate_button=True
).add_to(m)


# Think Folium Search got an update
DBRA_search = plugins.Search(
    position='bottomleft',
    layer = DBRA, 
    geom_type = 'Polygon',
    placeholder='DBRA Projects',
    search_label = "names", 
    search_zoom=17).add_to(m)

# .... I don't have a good reason to have this other than it is pretty cool.
minimap = plugins.MiniMap(position = 'bottomleft')
m.add_child(minimap)




m

In [57]:
m.save('Search_DBRA.html')

In [36]:
help(folium.plugins.search)

Help on module folium.plugins.search in folium.plugins:

NAME
    folium.plugins.search - # -*- coding: utf-8 -*-

CLASSES
    branca.element.MacroElement(branca.element.Element)
        Search
    
    class Search(branca.element.MacroElement)
     |  Adds a search tool to your map.
     |  
     |  Parameters
     |  ----------
     |  layer: GeoJson, TopoJson, FeatureGroup, MarkerCluster class object.
     |      The map layer to index in the Search view.
     |  search_label: str, optional
     |      'properties' key in layer to index Search, if layer is GeoJson/TopoJson.
     |  search_zoom: int, optional
     |      Zoom level to set the map to on match.
     |      By default zooms to Polygon/Line bounds and points
     |      on their natural extent.
     |  geom_type: str, default 'Point'
     |      Feature geometry type. "Point", "Line" or "Polygon"
     |  position: str, default 'topleft'
     |      Change the position of the search bar, can be:
     |      'topleft', 'to

In [41]:
help(folium.plugins.minimap)

Help on module folium.plugins.minimap in folium.plugins:

NAME
    folium.plugins.minimap - # -*- coding: utf-8 -*-

CLASSES
    branca.element.MacroElement(branca.element.Element)
        MiniMap
    
    class MiniMap(branca.element.MacroElement)
     |  Add a minimap (locator) to an existing map.
     |  
     |  Uses the Leaflet plugin by Norkart under BSD 2-Clause "Simplified" License.
     |  https://github.com/Norkart/Leaflet-MiniMap
     |  
     |  Parameters
     |  ----------
     |  tile_layer : folium TileLayer object or str, default None
     |      Provide a folium TileLayer object or the wanted tiles as string.
     |      If not provided it will use the default of 'TileLayer', currently
     |      OpenStreetMap.
     |  position : str, default 'bottomright'
     |      The standard Control position parameter for the widget.
     |  width : int, default 150
     |      The width of the minimap in pixels.
     |  height : int, default 150
     |      The height of the m